In [24]:
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy import stats

In [7]:
####### Crear la capa en QGIS, con el siguiente código

from qgis.core import (
    QgsProject,
    QgsVectorLayer,
    QgsFeatureRequest,
    QgsProcessingFeedback,
    QgsField,
    QgsVectorFileWriter
)
from PyQt5.QtCore import QVariant
import processing

# Obtener la capa activa
urban_layer = iface.activeLayer()
if not urban_layer or not urban_layer.isValid():
    print("Error al obtener la capa activa")
    exit()

# Verificar el nombre del campo NOM_MET
field_names = [field.name() for field in urban_layer.fields()]
if 'NOM_MET' not in field_names:
    print("La capa activa no tiene el campo 'NOM_MET'. Campos disponibles:", field_names)
    exit()
    
# Crear una capa vacía para las cuadrículas combinadas
grid_layer = QgsVectorLayer("Polygon?crs=EPSG:6372", "Combined Grid", "memory")
grid_provider = grid_layer.dataProvider()

# Agregar el campo NOM_MET a la capa de cuadrícula combinada
grid_provider.addAttributes([QgsField("NOM_MET", QVariant.String)])
grid_layer.updateFields()

# Obtener los nombres de las ciudades
cities = urban_layer.uniqueValues(urban_layer.fields().indexOf("NOM_MET"))

# Parámetros de la cuadrícula hexagonal
hex_size = 1200  # Tamaño del hexágono en metros

# Feedback para el procesamiento
feedback = QgsProcessingFeedback()

for city in cities:
    # Filtrar la capa por ciudad
    urban_layer.selectByExpression(f'"NOM_MET" = \'{city}\'')
    city_layer = urban_layer.materialize(QgsFeatureRequest().setFilterFids(urban_layer.selectedFeatureIds()))

    # Generar la cuadrícula hexagonal
    grid_extent = city_layer.extent()
    grid_params = {
        'TYPE': 4,  # Hexágonos
        'EXTENT': grid_extent,
        'HSPACING': hex_size,
        'VSPACING': hex_size,
        'CRS': city_layer.crs().toWkt(),
        'OUTPUT': 'memory:'
    }
    hex_grid = processing.run("qgis:creategrid", grid_params, feedback=feedback)['OUTPUT']
    
    # Seleccionar hexágonos que intersectan con la mancha urbana
    select_by_location_params = {
        'INPUT': hex_grid,
        'PREDICATE': [0],  # Intersects
        'INTERSECT': city_layer,
        'METHOD': 0  # Create new selection
    }
    processing.run("qgis:selectbylocation", select_by_location_params, feedback=feedback)
    
    # Materializar la selección
    intersecting_hexagons = hex_grid.materialize(QgsFeatureRequest().setFilterFids(hex_grid.selectedFeatureIds()))
    
    # Añadir atributo NOM_MET a los hexágonos
    with edit(grid_layer):
        for feature in intersecting_hexagons.getFeatures():
            new_feature = QgsFeature(grid_layer.fields())
            new_feature.setGeometry(feature.geometry())
            new_feature['NOM_MET'] = city
            grid_provider.addFeatures([new_feature])
        
# Guardar la capa combinada en un archivo
output_path = "C:/Users/ADMIN/Desktop/Ejercicio/Factores Exógenos/Cuadrícula Hexagonal1600.shp"  # Cambia esto a la ruta de tu archivo de salida
QgsVectorFileWriter.writeAsVectorFormat(grid_layer, output_path, "UTF-8", grid_layer.crs(), "ESRI Shapefile")

# Añadir la capa combinada al proyecto
QgsProject.instance().addMapLayer(grid_layer)

print("Proceso completado.")




###### Proceso en Python

# Cargar la capa de hexágonos
hex_grid = gpd.read_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Cuadrícula Hexagonal1600.shp")

In [ ]:
# Cargar la capa de manzanas
manzanas = gpd.read_file(r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Tipologías\Nacional\240527_Mapa_Subtipologías_PCU.gpkg", layer="Mapa_SubTipologias_PCUS")

In [8]:
# Crear un campo ID único para cada hexágono
hex_grid['hex_grid_id'] = range(1, len(hex_grid) + 1)

# Asegurar que ambas capas tengan el mismo sistema de referencia espacial (CRS)
if hex_grid.crs != manzanas.crs:
    manzanas = manzanas.to_crs(hex_grid.crs)

# Realizar la intersección espacial entre hexágonos y manzanas
intersected = gpd.overlay(manzanas, hex_grid, how='intersection')

# Agrupar por los identificadores de los hexágonos
grouped = intersected.groupby('hex_grid_id')  # Cambia 'hex_grid_id' al nombre correcto del campo ID de los hexágonos

In [9]:
# Crear las nuevas columnas para sumatorias en la capa de hexágonos
sum_fields = [
    'POPTOT2010', 'POPTOT2020', 'VIVTOT2010', 'VIVTOT2020',
    'VIVPARDES2010', 'VIVPARDES2020', 'area_ha',
    'CUARTOS2010', 'CUARTOS2020'
]

for field in sum_fields:
    hex_grid[f'sum_{field}'] = 0

# Crear una columna para la cantidad de manzanas que cruzan
hex_grid['manzanas_count'] = 0

# Calcular las sumatorias y la cantidad de manzanas
for hex_id, group in grouped:
    for field in sum_fields:
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'sum_{field}'] = group[field].sum()
    
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'manzanas_count'] = len(group)

In [10]:
# Calcular las modas y porcentajes (8 para TIPOLOGIA y 4 para R_DESHAB)
for i in range(1, 9):  # 8 categorías para TIPOLOGIA
    hex_grid[f't_{i}'] = None
    hex_grid[f't_{i}_p'] = 0.0

for i in range(1, 5):  # 4 categorías para R_DESHAB
    hex_grid[f'r_{i}'] = None
    hex_grid[f'r_{i}_p'] = 0.0

In [11]:
# Calcular las modas y porcentajes
for hex_id, group in grouped:
    # Calcular para TIPOLOGIA
    top8_tipologia = group['TIPOLOGIA'].value_counts().head(8)
    total_count_tipologia = group['TIPOLOGIA'].count()

    for i, (category, count) in enumerate(top8_tipologia.items(), start=1):
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f't_{i}'] = category
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f't_{i}_p'] = (count / total_count_tipologia) * 100

    # Calcular para R_DESHAB
    top4_r_deshab = group['R_DESHAB'].value_counts().head(4)
    total_count_r_deshab = group['R_DESHAB'].count()

    for i, (category, count) in enumerate(top4_r_deshab.items(), start=1):
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'r_{i}'] = category
        hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, f'r_{i}_p'] = (count / total_count_r_deshab) * 100

In [13]:
# Guardar el resultado en un archivo GPKG
hex_grid.to_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Cuadrícula_Hexagonal_Actualizada.gpkg", driver="GPKG")

Indicadores

In [10]:
# Cargar la capa de hexágonos
hex_grid = gpd.read_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Cuadrícula_Hexagonal_Actualizada.gpkg")

In [11]:
# Eliminar el prefijo 'sum_' de los nombres de las columnas
hex_grid.columns = hex_grid.columns.str.replace('^sum_', '', regex=True)

In [12]:
# Reemplazar valores NaN por 0
hex_grid = hex_grid.fillna(0)

# Calcular los nuevos campos
hex_grid['difpob'] = hex_grid['POPTOT2020'] - hex_grid['POPTOT2010']
hex_grid['relpob'] = hex_grid['difpob'] / hex_grid['POPTOT2010']
hex_grid['tasapob'] = (hex_grid['POPTOT2020'] / hex_grid['POPTOT2010'])**(1/10) - 1

hex_grid['denspop10'] = hex_grid['POPTOT2010'] / hex_grid['area_ha']
hex_grid['denspop20'] = hex_grid['POPTOT2020'] / hex_grid['area_ha']

hex_grid['difviv'] = hex_grid['VIVTOT2020'] - hex_grid['VIVTOT2010']
hex_grid['relviv'] = hex_grid['difviv'] / hex_grid['VIVTOT2010']
hex_grid['tasaviv'] = (hex_grid['VIVTOT2020'] / hex_grid['VIVTOT2010'])**(1/10) - 1

hex_grid['densviv10'] = hex_grid['VIVTOT2010'] / hex_grid['area_ha']
hex_grid['densviv20'] = hex_grid['VIVTOT2020'] / hex_grid['area_ha']

hex_grid['habxviv10'] = hex_grid['POPTOT2010'] / hex_grid['VIVTOT2010']
hex_grid['habxviv20'] = hex_grid['POPTOT2020'] / hex_grid['VIVTOT2020']

hex_grid['difcuartos'] = hex_grid['CUARTOS2020'] - hex_grid['CUARTOS2010']
hex_grid['relcuartos'] = hex_grid['difcuartos'] / hex_grid['CUARTOS2010']

hex_grid['cperc10'] = hex_grid['CUARTOS2010'] / hex_grid['POPTOT2010']
hex_grid['cperc20'] = hex_grid['CUARTOS2020'] / hex_grid['POPTOT2020']

hex_grid['cperviv10'] = hex_grid['CUARTOS2010'] / hex_grid['VIVTOT2010']
hex_grid['cperviv20'] = hex_grid['CUARTOS2020'] / hex_grid['VIVTOT2020']

hex_grid['deshabitacion'] = hex_grid['VIVPARDES2020'] / hex_grid['VIVTOT2020']

# Guardar el resultado en un nuevo archivo GPKG
hex_grid.to_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\240929_factores_exógenos_deshabitación.gpkg", driver="GPKG")

Preparar Censo

In [16]:
# Cargar la capa de manzanas
manzanas = gpd.read_file(r"C:\Users\ADMIN\Desktop\Ejercicio\MZN_NAC")

# Cargar los datos censales para 2010 y 2020
censo_2010 = pd.read_csv(r"C:\Users\ADMIN\Desktop\Ejercicio\Base de Datos Censal\MZNA_2010_NAC.csv", usecols=['CVEGEO', 'PEA'])
censo_2020 = pd.read_csv(r"C:\Users\ADMIN\Desktop\Ejercicio\Base de Datos Censal\MZNA_2020_NAC.csv", usecols=['CVEGEO', 'PEA'])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19044\4156862868.py:5: DtypeWarning: Columns (132) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_2010 = pd.read_csv(r"C:\Users\ADMIN\Desktop\Ejercicio\Base de Datos Censal\MZNA_2010_NAC.csv", usecols=['CVEGEO', 'PEA'])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19044\4156862868.py:6: DtypeWarning: Columns (142) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_2020 = pd.read_csv(r"C:\Users\ADMIN\Desktop\Ejercicio\Base de Datos Censal\MZNA_2020_NAC.csv", usecols=['CVEGEO', 'PEA'])


In [18]:
# Función para reemplazar '*' con '1' en el DataFrame
def replace(dataframe):
    return dataframe.replace('*', '1')

# Aplicar la función replace para reemplazar '*' con '1'
censo_2010 = replace(censo_2010)
censo_2020 = replace(censo_2020)

# Convertir las columnas PEA a numérico (por si hay valores no numéricos después de la sustitución)
censo_2010['PEA'] = pd.to_numeric(censo_2010['PEA'], errors='coerce')
censo_2020['PEA'] = pd.to_numeric(censo_2020['PEA'], errors='coerce')

# Renombrar las columnas PEA para distinguir los años
censo_2010.rename(columns={'PEA': 'PEA2010'}, inplace=True)
censo_2020.rename(columns={'PEA': 'PEA2020'}, inplace=True)

# Realizar la unión por el campo CVEGEO
manzanas = manzanas.merge(censo_2010, on='CVEGEO', how='left')
manzanas = manzanas.merge(censo_2020, on='CVEGEO', how='left')

# Reemplazar valores NaN en los campos PEA2010 y PEA2020 con 0
manzanas[['PEA2010', 'PEA2020']] = manzanas[['PEA2010', 'PEA2020']].fillna(0)

# Cargar la capa de hexágonos
hex_grid = gpd.read_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\240929_factores_exógenos_deshabitación.gpkg")

# Asegurar que ambas capas tengan el mismo sistema de referencia espacial (CRS)
if hex_grid.crs != manzanas.crs:
    manzanas = manzanas.to_crs(hex_grid.crs)

# Realizar la intersección espacial entre hexágonos y manzanas
intersected = gpd.overlay(manzanas, hex_grid, how='intersection')

# Agrupar por los identificadores de los hexágonos
grouped = intersected.groupby('hex_grid_id')

# Crear las nuevas columnas para sumatorias de PEA en la capa de hexágonos
hex_grid['PEA2010'] = 0
hex_grid['PEA2020'] = 0

# Calcular las sumatorias y la cantidad de manzanas
for hex_id, group in grouped:
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'PEA2010'] = group['PEA2010'].sum()
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'PEA2020'] = group['PEA2020'].sum()
    

In [19]:
hex_grid['depeco10'] = 1-(hex_grid['PEA2010'] / hex_grid['POPTOT2010'])
hex_grid['depeco20'] = 1-(hex_grid['PEA2020'] / hex_grid['POPTOT2020'])

In [21]:
# Guardar el resultado en un nuevo archivo GPKG
hex_grid.to_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\240929_factores_exógenos_deshabitación_v1_PEA.gpkg", driver="GPKG")

NSE

In [22]:
# Cargar la capa de AGEB y el archivo de NSE
ageb_layer = gpd.read_file(r"C:\Users\ADMIN\Desktop\Censos\shp\MG_2020_Integrado\conjunto_de_datos\00a.shp")
nse_data = pd.read_excel(r"C:\Users\ADMIN\Desktop\Ejercicio\NSE\nse_ageb_amai.xlsx", usecols=['CVEAGEB', 'PONDERADO', 'PREDOMINANTE'])

In [ ]:
# Hacer join entre nse_data y la capa AGEB por los campos CVEAGEB y CVEGEO
ageb_layer = ageb_layer.merge(nse_data, left_on='CVEGEO', right_on='CVEAGEB', how='left')

# Asegurar que ambas capas tengan el mismo sistema de referencia espacial (CRS)
if hex_grid.crs != ageb_layer.crs:
    ageb_layer = ageb_layer.to_crs(hex_grid.crs)

# Realizar la intersección espacial entre hexágonos y AGEB
intersected = gpd.overlay(ageb_layer, hex_grid, how='intersection')

# Calcular el área de la intersección para usarla como ponderador
intersected['intersect_area'] = intersected.area

# Agrupar por los identificadores de los hexágonos
grouped = intersected.groupby('hex_grid_id')

# Crear nuevas columnas en hex_grid para el promedio ponderado de PONDERADO y la moda de PREDOMINANTE
hex_grid['ponderado_media'] = np.nan
hex_grid['predominante_moda'] = np.nan

In [25]:
# Función para calcular promedio ponderado
def weighted_average(data, values, weights):
    return np.average(data[values], weights=data[weights])

# Iterar por los grupos y calcular el promedio ponderado y la moda
for hex_id, group in grouped:
    # Promedio ponderado de PONDERADO
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'ponderado_media'] = weighted_average(group, 'PONDERADO', 'intersect_area')
    
    # Moda de PREDOMINANTE
    hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'predominante_moda'] = stats.mode(group['PREDOMINANTE'])[0][0]

# Guardar el resultado en un nuevo archivo GPKG
hex_grid.to_file(r"C:\Users\ADMIN\Desktop\Ejercicio\Factores Exógenos\Hexagonos_NSE.gpkg", driver="GPKG")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19044\3076498455.py:11: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'predominante_moda'] = stats.mode(group['PREDOMINANTE'])[0][0]
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19044\3076498455.py:11: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  hex_grid.loc[hex_grid['hex_grid_id'] == hex_id, 'predominante_moda'] = stats.mode(group['PREDOM